# Короткие посты: тетрадка 0

"Почему бы не попробовать автомазировать создание коротких постов?" - подумал я, и решил сделать создание постов наподобие "самые-самые учатся у нас" более автоматическим.

Первой тетрадкой будет тетрадка для добычи постов из группы и поиском кандидаток для таких постов.

// данная и последующие тетрадки <br>
// преследуют лишь образовательные цели, <br>
// в частности, для демонстрации возможностей <br> 
// библиотек языка Python

## Добываем посты для проекта!

Для получения постов из группы используем эмулятор браузера Selenium.

In [1]:
from selenium import webdriver

driver = webdriver.Chrome()#'path to chromedriver')

In [2]:
driver.get('https://vk.com/imit_omsu')

In [3]:
search = driver.find_element_by_class_name('ui_tab_search')

In [4]:
search.click()

In [5]:
row = driver.find_element_by_id('wall_search')

In [6]:
row.clear()
row.send_keys('самые красивые девушки\n')
#row.clear()
#row.send_keys('девушка месяца\n')

Скроллим до конца!..

In [7]:
%%time
from selenium.webdriver.common.keys import Keys
import time
import numpy as np

def scroll_to_the_end(driver=driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        pause = np.abs(np.random.normal()) + 0.2
        time.sleep(pause)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
scroll_to_the_end()

CPU times: user 318 ms, sys: 473 ms, total: 791 ms
Wall time: 645 ms


Чтобы получить html-разметку страницы будем использовать чудесный суп.

In [11]:
from bs4 import BeautifulSoup
bs = BeautifulSoup(driver.page_source, 'html.parser')

In [88]:
posts = bs.findAll('div', attrs={'class': 'post_content'})

In [89]:
def filter_posts_girls(posts):
    y = lambda x: str(x).lower()
    return [x for x in posts if u'девушк' in y(x) and u'красивые' in y(x) and u'мальчик' not in y(x)]

In [90]:
len(posts), len(filter_posts_girls(posts))

(40, 20)

In [91]:
girls = filter_posts_girls(posts)

In [92]:
girls[0].findAll('div', attrs={'class': 'wall_post_text'})

[<div class="wall_post_text">А ведь самые красивые девушки учатся у нас!</div>]

In [93]:
%%time
row.clear()
row.send_keys('девушка месяца\n')

time.sleep(2.3)
scroll_to_the_end()

bs = BeautifulSoup(driver.page_source, 'html.parser')

posts = bs.findAll('div', attrs={'class': 'post_content'})
def filter_posts_girls_month(posts):
    return [x for x in posts if u'девушк' in str(x).lower() and u'livejournal' in str(x).lower()]
len(filter_posts_girls_month(posts))

CPU times: user 2.06 s, sys: 2.12 ms, total: 2.06 s
Wall time: 7.94 s


Сохраним тексты постов и уберем посты с именами в тексте.

In [16]:
from natasha import NamesExtractor
extractor = NamesExtractor()

def get_texts(posts):
    t = []
    for x in posts:
        t.extend(list(map(lambda z: z.text, x.findAll('div', attrs={'class': 'wall_post_text'}))))
    t = [x for x in t if len(extractor(x).matches) == 0]
    return t


def save_texts(texts, name='data.txt'):
    with open(name, 'w') as f:
        f.write('\n'.join(texts))


def load_texts(name='data.txt'):
    with open(name) as f:
        return f.readlines()

In [95]:
texts = get_texts(girls)
save_texts(texts)

In [96]:
texts = load_texts()

Позже мы будем использовать эти тексты для генерации подписей для новых постов.

### Ищем героинь для нашего поста!

In [8]:
driver.get('https://vk.com/search?c%5Bgroup%5D=225037&c%5Bsection%5D=people')
sex = driver.find_element_by_xpath("//div[contains(@onclick,'radiobtn')]")
sex.click()

In [9]:
%%time
scroll_to_the_end()

CPU times: user 64.8 ms, sys: 3.26 ms, total: 68 ms
Wall time: 19.2 s


In [12]:
bs = BeautifulSoup(driver.page_source, 'html.parser')

def get_refs2persons(page):
    return ['vk.com' + x.findAll('a')[0].get_attribute_list('href')[0] for x in page.findAll('div', attrs={'class': 'labeled name'})]

In [13]:
candidates = get_refs2persons(bs)

In [14]:
len(candidates)

194

Пока не будем фильтровать список и сохраним кандидаток

In [17]:
save_texts(candidates, 'candidates.txt')

## Дополнительно добудем разные хвалебные эпитеты

In [102]:
from nltk import word_tokenize

In [106]:
driver.get('https://www.liveinternet.ru/users/4468278/post246073969/')

bs = BeautifulSoup(driver.page_source, 'html.parser')
compliments = [x for x in bs.findAll('p')]
compliments = [x for x in compliments if len(x) > 0]

In [107]:
compliments = [x.text.strip() for x in compliments]
compliments = [x for x in compliments if len(x) > 0]
compliments = [' '.join(word_tokenize(x)) for x in compliments]

In [109]:
import string
import nltk
from nltk.tokenize import RegexpTokenizer
import re

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def preprocess(sentence):
    tokenizer = RegexpTokenizer(r'[а-я]+|[А-Я]+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [morph.parse(w)[0].normal_form for w in tokens if morph.tag(w)[0].POS == 'ADJF']
    return filtered_words

In [111]:
tmp = []
for c in compliments:
    tmp.extend(preprocess(c))
tmp = list(set(tmp) ^ {
    'наш', 
    'свой', 
    'общий', 
    'непостоянный', 
    'беспечный', 
    'эгоистичный', 
    'сумасшедший', 
    'весь', 
    'каждый',
    'индивидуальный',
    'огнеопасный',
    'упоенный',
    'другой',
    'шипучий',
    'тот'
})

Печально, но не все эпитеты оказались положительной тональности.

Поэтому здесь имеет смысл использовать классификатор тональности!

In [25]:
driver.close()

## Оффтоп: получим ссылки со страницы

In [26]:
import re
get_some_url = lambda x: re.findall(r'url\([^)(]+\)', str(x))
get_httplike = lambda x: [y[1:-1] for y in re.findall(r'[\"\']http.*[\"\']', str(x))]

In [27]:
#get_some_url(girls[0])

In [28]:
urls = list(map(get_some_url, girls))

In [29]:
#girls[8]

In [30]:
import numpy as np
counts = np.array(list(map(len, urls)))
#list(zip(counts == 0, range(len(counts))))

In [31]:
#get_httplike(girls[8])

In [32]:
#httplike = list(map(get_httplike, girls))
#counts = np.array(list(map(len, httplike)))
#list(zip(counts == 0, range(len(counts))))

In [33]:
#girls[8]